The particle dynamics in a general velocity field $ \mathbf{u}(\mathbf{x}, t) = \begin{pmatrix} u((\mathbf{x}, t)) \\ v((\mathbf{x}, t)) \end{pmatrix} $. This function evaluates the velocity field $ \mathbf{u}(\mathbf{x}, t) $, at point $ \mathbf{x} $ at time $ t $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| x | array (2,) | $ \mathbf{x} $ |
| X | array (Nx, Ny) | X-meshgrid|
| Y | array (Nx, Ny) | Y-meshgrid|
| Interpolant | list (2,) | Interpolant[0]: Interpolant object for $ u((\mathbf{x}, t))  $ <br /> Interpolant[1]: Interpolant object for $ v((\mathbf{x}, t))  $. <br /> <br /> The interpolation, however, works only <br /> if the particle is in the defined flow domain. <br /> At the boundary of the domain, <br /> the interpolant is not defined.|
| periodic | list (2,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br />|
| defined_domain | array (Nx, Ny) | points on the meshgrid where velocity field is defined |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| dt_data | float | time spacing of the velocity data |
| delta | list (2,) | delta[0]: x-grid spacing of the velocity data <br /> delta[1]: y-grid spacing of the velocity data|
| vx | float | $ u((\mathbf{x}, t)) $ |
| vy | float | $ v((\mathbf{x}, t)) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add Algorithm folder to current working path in order to access the functions
sys.path.append(parent_directory+"/utils")

In [2]:
# Import numpy
import numpy as np

# Import function that checks location of particle
from ipynb.fs.defs.check_location import check_location

# Import function that computes velocity in the domain
from ipynb.fs.defs.velocity_domain import velocity_domain

# Import function that computes velocity at the boundary
from ipynb.fs.defs.velocity_boundary import velocity_boundary

In [3]:
def velocity(t, x, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta):
    
    dx_data = delta[0]
    dy_data = delta[1]
    
    # check if periodic in x
    if periodic[0]:
        
        x[0] = x[0]%(X[0, -1, 0]-X[0, 0, 0])
    
    # check if periodic in y
    if periodic[1]:
        
        x[1] = x[1]%(Y[-1, 0, 0]-Y[0, 0, 0])
    
    # Check location of particle.
    loc, idx_x, idx_y = check_location(X, Y, defined_domain, x)
    
    # If particle inside defined domain
    # then velocity at (x, t)
    # can be evaluated using the interpolant
    if loc ==  "IN":
            
        vel = velocity_domain(t, x, Interpolant, dt_data, bool_unsteady)
            
        vx = vel[0]
        vy = vel[1]
            
        return [vx, vy]
    
    # otherwise we need evaluate the velocity using free-slip boundary conditions
    elif loc == "BOUNDARY":
            
        vel = velocity_boundary(t, x, idx_x, idx_y, defined_domain, X, Y, bool_unsteady, dt_data, dx_data, dy_data)
            
        vx = vel[0]
        vy = vel[1]
            
        return np.array([vx, vy])
        
    elif loc == "OUT":
            
        return [0, 0]